<a href="https://colab.research.google.com/github/NevroHelios/nn_practice/blob/main/Number_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dependency

In [ ]:
import torch
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Get data
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)
#1,28,28 - classes 0-9

In [ ]:
# Image Classifier Neural Network
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Instance of the neural network, loss, optimizer
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Training flow
if __name__ == "__main__":
    for epoch in range(10): # train for 10 epochs
        for batch in dataset:
            X,y = batch
            X, y = X.to('cuda'), y.to('cuda')
            yhat = clf(X)
            loss = loss_fn(yhat, y)

            # Apply backprop
            opt.zero_grad()
            loss.backward()
            opt.step()

        print(f"Epoch:{epoch} loss is {loss.item()}")

Epoch:0 loss is 0.028422188013792038
Epoch:1 loss is 0.004187010694295168
Epoch:2 loss is 0.0008276794687844813
Epoch:3 loss is 0.0008882067631930113
Epoch:4 loss is 6.617339386139065e-05
Epoch:5 loss is 1.686500218056608e-05
Epoch:6 loss is 0.00027072662487626076
Epoch:7 loss is 0.00012716386117972434
Epoch:8 loss is 1.254431754205143e-05
Epoch:9 loss is 1.8626434439283912e-07


In [ ]:
from torchvision.transforms import Grayscale
import os
with open('model_state.pt', 'rb') as f:
    clf.load_state_dict(load(f))

img = 'img_1,7,3,8.png'
dir = '/resized_images/'
os.makedirs(dir, exist_ok= True)
path = os.path.join(dir, f'resized_{img}')

img_colored = Image.open(img)
img_gray = Grayscale()(img_colored)

resized_image = img_gray.resize((28, 28), Image.ANTIALIAS)
resized_image.save(path)

img_tensor = ToTensor()(resized_image).unsqueeze(0).to('cuda')

print(torch.argmax(clf(img_tensor)))


tensor(8, device='cuda:0')


<ipython-input-27-95fe47ff3db6>:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = img_gray.resize((28, 28), Image.ANTIALIAS)
